In [9]:
import cv2
import numpy as np
import onnxruntime as ort

In [10]:
model_path = r".\model_quantized.onnx"

In [11]:
video_path=r'.\violentVideo\vid_1.mp4'

In [12]:
def img_preprocessing(img):
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    for i in range(3):
        img[:, :, i] = (img[:, :, i] - mean[i]) / std[i]
    img = np.transpose(img, (2, 0, 1))
    return img

In [13]:
def predict_on_video(video_path,onnx_model_path,n_fps_pred=7,n_frames_pred=8,n_frames_split=1):
    #loader model
    
    session = ort.InferenceSession(onnx_model_path)
    input_name=session.get_inputs()[0].name
    output_name=session.get_outputs()[0].name

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    fps=cap.get(cv2.CAP_PROP_FPS)
    interval=int(fps/n_fps_pred)
    X=[]
    violent_frames=[]
    tmp_violent_frames=[]
    counts=0
    while True:
        counts+=1
        ret,frame=cap.read()
        if not ret:
            break
        if(counts%interval==0):
            counts=0
            processed_frame=img_preprocessing(frame)
            X.append(processed_frame)
            tmp_violent_frames.append(frame)
        input_data=np.array([X.copy()],dtype=np.float32)
        if(len(input_data[0])!=n_frames_pred):
            pass
        else:
            outputs = session.run([output_name], {input_name: input_data})
            y_preds=np.argmax(outputs[0],axis=1)[0]
            # print(outputs[0][0])
            if(y_preds==1):
                if(len(violent_frames)>=n_frames_pred):
                    violent_frames.extend(tmp_violent_frames[-(n_frames_pred-int(n_frames_pred/n_frames_split)+1):])
                else:
                    violent_frames.extend(tmp_violent_frames)
                print(y_preds)
            X=X[-int(n_frames_pred/n_frames_split)+1:]
            tmp_violent_frames=tmp_violent_frames[-int(n_frames_pred/n_frames_split)+1:]
        cv2.imshow('Video',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        if not ret:
                break
    cap.release()
    cv2.destroyAllWindows()

    return violent_frames
        

In [14]:
n_fps_pred=8
n_frames_pred=10
violentFrames=predict_on_video(video_path=video_path,onnx_model_path=model_path,n_fps_pred=n_fps_pred,n_frames_pred=n_frames_pred)

[ 0.6341752  -0.85610455]
[ 0.68930155 -0.9068677 ]
[ 0.5170974 -0.7277536]
[ 0.41833454 -0.6269186 ]
[ 0.28243235 -0.48575974]
[ 0.22092387 -0.4190397 ]
[ 0.22115365 -0.42153266]
[ 0.3913206  -0.59581673]
[ 0.3324093  -0.53627825]
[ 0.38436338 -0.58982396]
[ 0.37155807 -0.5815867 ]
[ 0.25967318 -0.46276134]
[ 0.26713398 -0.47205532]
[ 0.24650273 -0.44956097]
[ 0.3576637 -0.5717567]
[ 0.42890653 -0.6427036 ]
[ 0.49942425 -0.7159916 ]
[ 0.61993515 -0.836775  ]
[ 0.7830252 -0.9970435]
[ 1.0387154 -1.2480308]
[ 0.9955965 -1.2019483]
[ 1.2954813 -1.4904777]
[ 0.816236 -1.011724]
[ 0.88246983 -1.0801685 ]
[ 0.80302227 -1.0033613 ]
[ 0.82538146 -1.0306473 ]
[ 0.8879144 -1.0979798]
[ 0.7927119 -1.0010618]
[ 0.8587642 -1.0717766]
[ 0.8328115 -1.0474324]
[ 0.8132427 -1.0283878]
[ 0.66965693 -0.8859466 ]
[ 0.7512188  -0.96417844]
[ 0.73675513 -0.9466669 ]
[ 0.6841843 -0.8908935]
[ 0.7547423 -0.9654823]
[ 0.69803244 -0.90890443]
[ 0.6257543 -0.8340409]
[ 0.5075913  -0.71431214]
[ 0.40051907 -0.60

In [15]:
np.array(violentFrames).shape

(27, 360, 640, 3)

In [16]:
for i in range(len(violentFrames)):
    # Show each frame
    # if(i>=12):
    try:

        cv2.imshow('Video Frame', violentFrames[i])
    except:
        print(violentFrames[i])
        break
    # else:
    #     cv2.imshow('Video Frame', violentFrames[i])
    
    # Wait for 25 ms between frames (40 FPS). Adjust as needed.
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break  # Press 'q' to exit early

cv2.destroyAllWindows()

# Save Video

In [17]:
if len(violentFrames)>0:
    height, width, layers = violentFrames[0].shape
    output_path = './Out_video/video_out.mp4'
    fps = n_fps_pred  # Số frame mỗi giây
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Định dạng mã hóa MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))


    for frame in violentFrames:
        out.write(frame)

 
    out.release()
    print("Video đã được lưu thành công tại", output_path)

Video đã được lưu thành công tại ./Out_video/video_out.mp4
